In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
store = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
store.head(30)

In [ ]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')

#200101
#1) train = train.drop([267730, 336495, 417801, 153916, 271300])
#default : axis=0

#2) train을 재정의 할때 -1100 보다 큰 애들만 불러오기
train = train[train['Weekly_Sales'] > -1100]

train['Date']=train['Date'].astype('datetime64')
train['year']=train['Date'].dt.year
train['month']=train['Date'].dt.month
train['day'] =train['Date'].dt.day
train['week_nb'] = np.ceil(train['day'] / 7).astype('int64')
train['week'] =train['Date'].dt.week
train.head()


In [ ]:
train['Weekly_Sales'].sort_values()[:5].index

In [ ]:
#20.01.01

(train['Weekly_Sales'] < 0).sum()
# 특이치를 제거하지 않고 y값의 최소값을 구해서 최소값으로 값을 더하고 로그 취하고 나서 나중에 더해준 만큼 빼준다.

train['Weekly_Sales'].min()
# 최소값의 값이 너무 크기 때문에 데이터가 변형될수도 있음

# --> 너무 튀는 값들은 제거해버리고 많이 튀지 않은 값들은 최소값을 더해준다.(1,2번 을 합쳐서)

train['Weekly_Sales'].sort_values()[:20]


In [ ]:
train.groupby('year')['Weekly_Sales'].mean()
train.groupby('year')['Weekly_Sales'].median()
train['year'].value_counts()

import matplotlib.pyplot as plt
import seaborn as sns
train3 = train[train['Weekly_Sales'] < 50000]
a,b = plt.subplots(1,1,figsize=(20,12))
# sns.boxplot(train['month'],train3['Weekly_Sales'])
sns.distplot(train['Weekly_Sales'])

# 특이치가 너무 커져버리면 실제값과 예측값의 차이가 매우크면 오차의제곱이 커져버린다. -> 예측이 힘들어짐 -> 점수가 안나옴
# y값을 로그를 취해준다. -> 튀는값들이 많이 사라짐
# 특이치가 많아서 제거해주면 좋지만 정상적인 데이터 까지 날아가면 손실이 매우 크기때문에 로그를 취해준다.
# 음수가있으면 로그를 취할 수 없음.

In [ ]:
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
test.head()
test['Date']=test['Date'].astype('datetime64')
test['year']=test['Date'].dt.year
test['year'].value_counts()
test['month']=test['Date'].dt.month
test['day'] =test['Date'].dt.day
test['week_nb'] = np.ceil(test['day'] / 7).astype('int64')
test['week'] =test['Date'].dt.week

In [ ]:
train = pd.merge(train,store,on='Store',how='left')
test = pd.merge(test,store,on='Store',how='left')
train.head()

In [ ]:
train.groupby('Type')['Weekly_Sales'].mean()
train = train.replace({'A': 1, 'B':2 ,'C':3})
test = test.replace({'A': 1, 'B':2 ,'C':3})

In [ ]:
train4 = train[train['Weekly_Sales'] < 50000]
a,b = plt.subplots(1,1,figsize=(20,12))
sns.boxplot(train['week'],train4['Weekly_Sales'])

In [ ]:
train2 = train.drop(['Date','Weekly_Sales'],1)
train2.head()


In [ ]:
test2 = test.drop(['Date'],axis=1)
test2.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, n_jobs=-1,random_state=1) # 기본값이 10으로 설정되어있으니 데이터셋이 복잡해짐에따라 esimators 값을 변경해준다
rf.fit(train2,np.log(train['Weekly_Sales']+1100))
prediction = rf.predict(test2)

In [ ]:
sub = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
sub.head()

In [ ]:
sub['Weekly_Sales'] = np.exp(prediction)-1100
sub.head()

In [ ]:
sub.to_csv('walmart.csv',index=False)

In [ ]:
df = pd.DataFrame(rf.feature_importances_)
df['col_name'] = train2.columns
df = df.sort_values(0,ascending=False)
df